###  Lab 11
-Based on previous task 'PhoneBook' implement following, using functions and stored procedures:
1.Function that returns all records based on a pattern (example of pattern: part of name, surname, phone number)

2.Create procedure to insert new user by name and phone, update phone if user already exists

3.Create procedure to insert many new users by list of name and phone. Use loop and if statement in stored procedure. Check correctness of phone in procedure and return all incorrect data.

4.Create function to querying data from the tables with pagination (by limit and offset)

5.Implement procedure to deleting data from tables by username or phone

Create database(phonebook_db11) and table(phonebook):  '''CREATE DATABASE phonebook_db11;'''  \\\ '''CREATE TABLE phonebook(
            id SERIAL PRIMARY KEY,
            username VARCHAR(50) NOT NULL,
            phone VARCHAR(20) NOT NULL,
            surname VARCHAR(50) NOT NULL
            );'''

###1 pattern (function): ''' CREATE OR REPLACE FUNCTION search_pattern(pattern TEXT)
RETURNS TABLE (
    id INT,
    username VARCHAR(50),
    surname VARCHAR(50),
    phone VARCHAR(50)
)
AS $$
BEGIN
    RETURN QUERY
    SELECT phonebook.id, phonebook.username, phonebook.surname, phonebook.phone
    FROM phonebook
    WHERE phonebook.username ILIKE '%' || pattern || '%'
       OR phonebook.surname ILIKE '%' || pattern || '%'
       OR phonebook.phone ILIKE '%' || pattern || '%';
END;
$$ LANGUAGE plpgsql;   '''




('''SELECT * FROM search_pattern('Yenleak'); ''')

###2 insert or update(procedure): '''  CREATE OR REPLACE PROCEDURE insert_or_update_user(uname TEXT, sname TEXT, ph TEXT)
AS $$
BEGIN
    IF EXISTS (SELECT 1 FROM phonebook WHERE username = uname AND surname = sname) THEN
        UPDATE phonebook SET phone = ph
        WHERE username = uname AND surname = sname;
    ELSE
        INSERT INTO phonebook(username, surname, phone)
        VALUES (uname, sname, ph);
    END IF;
END;
$$ LANGUAGE plpgsql;  '''



('''CALL insert_or_update_user('Ali', 'Nurlanov', '+77012345678');''')

###3 insert many users(procedure): ''' CREATE OR REPLACE PROCEDURE insert_many_users(users TEXT[][])
LANGUAGE plpgsql
AS $$
DECLARE
    i INT := 1;
    uname TEXT;
    sname TEXT;
    ph TEXT;
BEGIN
    CREATE TEMP TABLE IF NOT EXISTS invalid_data (username TEXT, surname TEXT, phone TEXT);

    WHILE i <= array_length(users, 1) LOOP
        uname := users[i][1];
        sname := users[i][2];
        ph := users[i][3];

        IF ph ~ '^\+?[0-9]{10,15}$' THEN
            CALL insert_or_update_user(uname, sname, ph);
        ELSE
            INSERT INTO invalid_data VALUES (uname, sname, ph);
        END IF;
        i := i + 1;
    END LOOP;

    -- (cоңында қате деректерді қайтарамыз)
    RAISE NOTICE 'Invalid data:';
    FOR uname, sname, ph IN SELECT * FROM invalid_data LOOP
        RAISE NOTICE 'Invalid: %, %, %', uname, sname, ph;
    END LOOP;
END;
$$;   '''


(''' CALL insert_many_users(ARRAY[
    ARRAY['Arman', 'Tleujan', '87051234567'],
    ARRAY['Didar', 'Serikov', '87001112233'],
    ARRAY['Altynai', 'Zhumakhan', 'not-a-number']
]);
 ''')

###4 query with pagination(function): ''' CREATE OR REPLACE FUNCTION get_users_paginated(lim INT, off INT)
RETURNS TABLE(id INT, username TEXT, surname TEXT, phone TEXT)
AS $$
BEGIN
    RETURN QUERY
    SELECT * FROM phonebook ORDER BY id LIMIT lim OFFSET off;
END;
$$ LANGUAGE plpgsql;    '''


(''' SELECT * FROM get_users_paginated(4, 0);  -- алғашқы 1-бет
SELECT * FROM get_users_paginated(4, 4);  -- келесі 2-бет
 ''')

###5 delete user(prosedure):  ''' CREATE OR REPLACE PROCEDURE delete_user(u TEXT)
AS $$
BEGIN
    DELETE FROM phonebook WHERE username = u OR phone = u;
END;
$$ LANGUAGE plpgsql;  '''


('''  CALL delete_user('Alikhan');  ''')